In [1]:
%load_ext autoreload
%autoreload 2
%cd '/home/xingyu/Projects/PlasticineLab'
%pwd

import numpy as np
from imitation.agent import Agent, sample_traj
from plb.envs import make
from imitation.train import get_args
from plb.engine.taichi_env import TaichiEnv
from plb.optimizer.solver import Solver
from plb.algorithms.logger import Logger
device = 'cuda'

log_dir = './data/connect'
args = get_args("")

obs_channel = len(args.img_mode)
img_obs_shape = (args.image_dim, args.image_dim, obs_channel)


env = make(args.env_name, nn=(args.algo == 'nn'), sdf_loss=args.sdf_loss,
           density_loss=args.density_loss, contact_loss=args.contact_loss,
           soft_contact_loss=args.soft_contact_loss, chamfer_loss=args.chamfer_loss)
env.seed(args.seed)
taichi_env: TaichiEnv = env.unwrapped.taichi_env
T = env._max_episode_steps
action_dim = taichi_env.primitives.action_dim

plb_logger = Logger(log_dir)
solver = Solver(taichi_env, plb_logger, None,
                n_iters=(args.gd_num_steps + T - 1) // T, softness=args.softness, horizon=T,
                **{"optim.lr": args.lr, "optim.type": args.optim, "init_range": 0.0001})
agent = Agent(args, solver, img_obs_shape, action_dim, num_tools=2, device=device)

/home/xingyu/Projects/PlasticineLab
[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-6hzzblan
[Taichi] version 0.7.15, llvm 10.0.0, commit cff542ce, linux, python 3.6.9
[Taichi] Starting on arch=cuda
Building primitive
action:
  dim: 6
  scale: (0.7, 0.005, 0.005, 0.005, 0.0, 0.0)
color: (0.8, 0.8, 0.8)
friction: 0.9
h: 0.3
init_pos: (0.55, 0.2, 0.5)
init_rot: (0.707, 0.707, 0.0, 0.0)
lower_bound: (0.0, 0.09, 0.0)
r: 0.03
shape: RollingPinExt
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 6
  scale: (0.015, 0.0, 0.0, 0.0, 0.0, 0.0)
color: (0.8, 0.8, 0.8)
friction: 0.9
init_pos: (0.7, 0.1, 0.5)
init_rot: (0.707, 0.707, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
shape: Box
size: (0.015, 0.15, 0.1)
upper_bound: (1.0, 1.0, 1.0)
variations: None
{'init_pos': (0.5, 0.03576, 0.5), 'width': (0.2, 0.03575, 0.2), 'color': 100}
Initialize Renderer
bake_size: 6  
camera_pos: (0.5, 0.4, 1.9)  
camera_rot: (0.2, 0.0)  
dx: 0.006666666666666667  
image_res:

In [4]:
from imitation.utils import load_target_info
target_info = load_target_info(args, device)
np_target_imgs, target_imgs, np_target_mass_grids = target_info['np_target_imgs'], target_info['target_imgs'], target_info['np_target_mass_grids']

In [3]:
import torch
import matplotlib.pyplot as plt
from imitation.utils import img_to_np, write_number, img_to_tensor
from plb.utils.visualization_utils import make_grid, save_rgb
import os.path as osp
import sys

sys.path.insert(0, '/home/xingyu/Projects/PlasticineLab/')

# Trained on hindsight goals
# agent_path = 'data/local/0629_PushSpread_train_fea/0629_PushSpread_train_fea_2021_07_01_11_34_12_0001/agent_350.ckpt'
# Trained on real goals
agent_path = 'data/local/0722_PushSpread_train_fea_aug/0722_PushSpread_train_fea_aug_2021_07_23_12_12_33_0001/agent_50.ckpt'
agent = Agent(args, solver, img_obs_shape, action_dim, num_tools=2, device=device)
agent.feas.eval()
agent.load(agent_path)
taichi_env.loss.set_target_update(False)

agent_dataset = './data/autobot/0718_PushSpread_train_fea/0718_PushSpread_train_fea/0718_PushSpread_train_fea_2021_07_20_11_34_11_0001/agent_dataset.gz'
from imitation.buffer import ReplayBuffer
buffer = ReplayBuffer()
buffer.load(agent_dataset)

class Agent: obs_shape:  (64, 64, 8)
Agent loaded from data/local/0722_PushSpread_train_fea_aug/0722_PushSpread_train_fea_aug_2021_07_23_12_12_33_0001/agent_50.ckpt
Loading dataset in ./data/autobot/0718_PushSpread_train_fea/0718_PushSpread_train_fea/0718_PushSpread_train_fea_2021_07_20_11_34_11_0001/agent_dataset.gz
Loading dataset from ./data/autobot/0718_PushSpread_train_fea/0718_PushSpread_train_fea/0718_PushSpread_train_fea_2021_07_20_11_34_11_0001/agent_dataset.gz


In [15]:
import json
from imitation.encoder.vae import VAE
vae_dir = 'data/autobot/0729_vae/0729_vae/0729_vae_2021_07_29_15_53_40_0006/'
vae_path = osp.join(vae_dir, 'encoder_95.pth')
vae_vv_path = osp.join(vae_dir, 'variant.json')
with open(vae_vv_path, 'r') as f:
    vae_vv = json.load(f)
vae = VAE(4)
vae.load_state_dict(torch.load(vae_path))
vae = vae.cuda()

In [13]:
# Test reconstruction of VAE
reconst = vae.reconstr(target_imgs)

torch.Size([200, 4, 64, 64])


In [16]:
from imitation.compose_skills import plan, visualize_all_traj
test_init_v = np.array([95, 95, 95, 91, 91, 91, ])
test_target_v = np.array([25, 195, 128, 21, 191, 128])

for init_v, target_v in zip(test_init_v, test_target_v):
    plan_info = {'env':env, 'vae':vae,
                 'init_v':init_v, 'target_v': target_v, 'num_sample': 100}
    plan_info.update(**target_info)
    imgs = plan(agent, plan_info, opt_mode='sample')
    # best_traj, all_traj = plan(agent, plan_info, opt_mode='sample')
    # img = visualize_all_traj(all_traj)
    break

{'init_pos': (0.5, 0.03576, 0.5), 'width': (0.2, 0.03575, 0.2), 'color': 100}
initialize: [0.5 0.4 1.9] [0.2 0. ]


In [17]:
cpu_imgs = img_to_np(imgs)
grid_img = make_grid(cpu_imgs, ncol=20)
save_rgb('./data/connect/vae_samples.png', grid_img)